In [1]:
import dill as pickle
import numpy as np

In [2]:

def evaluate_combination(list_of_models):

    with open('data_splits.pickle', 'rb') as f:
        data_splits = pickle.load(f)
    val_bounds = np.cumsum([0] + [len(data_splits[0][250]['normed'][fold_ix]) for fold_ix in range(5)])
    val_ids = [x for fold_ix in range(5) for x in data_splits[0][250]['normed'][fold_ix]]
    val0_ids = val_ids[::2]
    val1_ids = val_ids[1::2]

    def extract_entities(ps, n):
        START_WITH_I = True
        LOOK_AHEAD = True
        max_ps = ps.max(-1)

        ps = ps.argsort(-1)[...,::-1]
        # argmax
        cat_ps = ps[:, 0]
        # argmax2
        cat_ps2 = ps[:, 1]

        all_entities = {}
        new_entity = True
        current_cat = current_start = current_end = None

        # except for special tokens
        for ix in range(n):

            # logic on new entity
            if new_entity:
                # Background - ignore
                if cat_ps[ix] == 0:
                    pass

                # B-LABEL(1,3,5,7,...) - start entity
                elif cat_ps[ix] % 2 == 1:
                    current_cat = (cat_ps[ix] + 1) // 2
                    current_start = current_end = ix
                    new_entity = False

                    if current_cat in [6, 7]:
                        LOOK_AHEAD = False
                    else:
                        LOOK_AHEAD = True

                # I-LABEL(2,4,6,8,...) - conditional start
                elif cat_ps[ix] % 2 == 0:
                    if START_WITH_I:
                        # Condition: I-LABEL in argmax with B-LABEL in argmax2
                        if cat_ps[ix] == (cat_ps2[ix]+1):
                            current_cat = cat_ps[ix] // 2
                            current_start = current_end = ix
                            new_entity = False

                            if current_cat in [6, 7]:
                                LOOK_AHEAD = False
                            else:
                                LOOK_AHEAD = True

            # logic on ongoing entity
            else:
                # Background - save current entity and init current
                if cat_ps[ix] == 0:
                    if LOOK_AHEAD:
                        if ix < n - 1 and (cat_ps[ix+1] == current_cat*2) and (cat_ps2[ix] == current_cat*2):
                            current_end = ix
                        else:
                            # update current
                            if current_cat not in all_entities:
                                all_entities[current_cat] = []
                            all_entities[current_cat].append((current_start, current_end))

                            # init current for new start
                            new_entity = True
                            current_cat = current_start = current_end = None

                    else:
                        # update current
                        if current_cat not in all_entities:
                            all_entities[current_cat] = []
                        all_entities[current_cat].append((current_start, current_end))

                        # init current for new start
                        new_entity = True
                        current_cat = current_start = current_end = None

                # B-LABEL(1,3,5,7,...) - save current entity and start new
                elif cat_ps[ix] % 2 == 1:
                    if cat_ps[ix] == (current_cat*2-1):
                        # update current
                        if current_cat not in all_entities:
                            all_entities[current_cat] = []
                        all_entities[current_cat].append((current_start, current_end))

                        # start new current
                        current_cat = (cat_ps[ix] + 1) // 2
                        current_start = current_end = ix
                        new_entity = False

                        if current_cat in [6, 7]:
                            LOOK_AHEAD = False
                        else:
                            LOOK_AHEAD = True

                    else:
                        if LOOK_AHEAD:
                            if ix < n - 1 and (cat_ps[ix+1] == current_cat*2) and (cat_ps2[ix] == current_cat*2):
                                current_end = ix
                            else:
                                # update current
                                if current_cat not in all_entities:
                                    all_entities[current_cat] = []
                                all_entities[current_cat].append((current_start, current_end))

                                # start new current
                                current_cat = (cat_ps[ix] + 1) // 2
                                current_start = current_end = ix
                                new_entity = False

                                if current_cat in [6, 7]:
                                    LOOK_AHEAD = False
                                else:
                                    LOOK_AHEAD = True

                        else:
                            # update current
                            if current_cat not in all_entities:
                                all_entities[current_cat] = []
                            all_entities[current_cat].append((current_start, current_end))

                            # start new current
                            current_cat = (cat_ps[ix] + 1) // 2
                            current_start = current_end = ix
                            new_entity = False

                            if current_cat in [6, 7]:
                                LOOK_AHEAD = False
                            else:
                                LOOK_AHEAD = True

                # I-LABEL(2,4,6,8,...) - conditional continue
                elif cat_ps[ix] % 2 == 0:
                    # B-LABEL0, I-LABEL0 - continue
                    if cat_ps[ix] == current_cat*2:
                        current_end = ix
                    # B-LBAEL0, I-LABEL1 - conditional finish current entity
                    else:
                        if LOOK_AHEAD:
                            if ix < n - 1 and (cat_ps[ix+1] == current_cat*2) and (cat_ps2[ix] == current_cat*2):
                                current_end = ix
                            else:
                                # update current
                                if current_cat not in all_entities:
                                    all_entities[current_cat] = []
                                all_entities[current_cat].append((current_start, current_end))

                                # init current
                                new_entity = True
                                current_cat = current_start = current_end = None
                        else:
                            # update current
                            if current_cat not in all_entities:
                                all_entities[current_cat] = []
                            all_entities[current_cat].append((current_start, current_end))

                            # init current
                            new_entity = True
                            current_cat = current_start = current_end = None

        # last entity
        if not new_entity:
            # update current
            if current_cat not in all_entities:
                all_entities[current_cat] = []
            all_entities[current_cat].append((current_start, current_end))

        return all_entities

    def filter_ps(all_entities, ps, sample_index_map, sample_bounds):
        nonlocal word_len_filters
        nonlocal token_len_filters
        nonlocal score_filters
        nonlocal len_exp
        word_len_filters = [0] * 8

        new_entities = {}

        for cat_ix, min_num_tokens, min_num_words, min_score in zip(range(1, 8), token_len_filters, word_len_filters, score_filters):

            if cat_ix in all_entities:
                possible_entities = [x for x 
                                     in all_entities[cat_ix] 
                                     if x[2] >= min_num_tokens
                                     and x[1] > min_score]

                if cat_ix in (1, 2, 5):
                    if len(possible_entities) > 1:
                        max_score = -9999
                        for x in possible_entities:
                            entity_score = x[1]
                            if entity_score > max_score:
                                max_score = entity_score
                                biggest_entity = x
                        possible_entities = [biggest_entity]

                new_entities[cat_ix] = possible_entities

        return new_entities


    def first_token_merge(gather_a, gather_b, bounds_a, bounds_b, logits_a, logits_b, new_bounds):
        new_gather_a = [0]
        new_gather_b = [0]
        prev_same_a = False
        prev_same_b = False
        for ix in range(1, len(gather_a)):
            same_a = gather_a[ix] == gather_a[ix - 1]
            same_b = gather_b[ix] == gather_b[ix - 1]
            if same_a:
                assert not same_b
                if prev_same_b and not prev_same_a and new_gather_a[-1] != gather_a[ix]:
                    new_gather_a.append(gather_a[ix])
                    new_gather_b.append(gather_b[ix])
                else:
                    prev_same_a = same_a
                    prev_same_b = same_b
                    continue
            elif same_b:
                if prev_same_a and not prev_same_b and new_gather_b[-1] != gather_b[ix]:
                    new_gather_a.append(gather_a[ix])
                    new_gather_b.append(gather_b[ix])
                else:
                    prev_same_a = same_a
                    prev_same_b = same_b
                    continue
            else:
                new_gather_a.append(gather_a[ix])
                new_gather_b.append(gather_b[ix])
            prev_same_a = same_a
            prev_same_b = same_b
        new_bounds = []
        for index_tokens_a, index_tokens_b in zip(new_gather_a, new_gather_b):
            new_bounds.append((min(bounds_a[index_tokens_a, 0],
                                   bounds_b[index_tokens_b, 0]), 
                               max(bounds_a[index_tokens_a, 1],
                                   bounds_b[index_tokens_b, 1])))
        new_logits = logits_a[new_gather_a] + logits_b[new_gather_b]
        return new_logits, np.array(new_bounds)

    def merge_ps(logits_a, logits_b, bounds_a, bounds_b):
        a = bounds_a
        b = bounds_b
        mapping_a = []
        mapping_b = []
        new_bounds = []
        apos = 0
        bpos = 0
        a_s, a_e = a[apos]
        b_s, b_e = b[bpos]
        current_start = 0
        while True:
            if a_e == b_e:
                new_bounds.append((current_start, b_e))
                mapping_a.append(apos)
                mapping_b.append(bpos)
                if apos == len(a) - 1 or bpos == len(b) - 1:
                    break
                next_a_s, next_b_s = a[apos + 1][0], b[bpos + 1][0]
                if next_a_s < next_b_s:
                    apos += 1
                    a_s, a_e = a[apos]
                    current_start = a_s
                elif next_b_s < next_a_s:
                    bpos += 1
                    b_s, b_e = b[bpos]
                    current_start = b_s
                else:
                    apos += 1
                    bpos += 1
                    a_s, a_e = a[apos]
                    b_s, b_e = b[bpos]
                    current_start = a_s
            elif a_e < b_e:
                new_bounds.append((current_start, a_e))
                mapping_a.append(apos)
                mapping_b.append(bpos)
                apos += 1
                a_s, a_e = a[apos]
                current_start = a_s
            else:
                new_bounds.append((current_start, b_e))
                mapping_a.append(apos)
                mapping_b.append(bpos)
                bpos += 1
                b_s, b_e = b[bpos]
                current_start = b_s

        return first_token_merge(mapping_a, mapping_b, bounds_a, bounds_b, logits_a, logits_b, new_bounds)



    def process_sample(raw_ps, unfiltered_entities, index_map, bounds, gt_spans, num_tokens, match_stats):

        filtered_entities = filter_ps(unfiltered_entities, raw_ps, index_map, bounds)
        predicted_spans = {x: [map_span_to_word_indices(extend_tokens(span, num_tokens), index_map, bounds) for span, score, span_len in y] 
                           for x, y in filtered_entities.items()}
        token_spans = predicted_spans
        # token_spans = {x: y[0] for x, y in filtered_entities}

        false_positives = {x: [] for x in range(1, 8)}
        true_positives = {x: [] for x in range(1, 8)}
        num_entities = np.zeros(7)
        for cat_ix in range(1, 8):

            pspans = predicted_spans.get(cat_ix, [])
            gspans = gt_spans.get(cat_ix, [])
            num_entities[cat_ix - 1] = len(pspans)
            if not len(pspans) or not len(gspans):
                match_stats[cat_ix]['fn'] += len(gspans)
                match_stats[cat_ix]['fp'] += len(pspans)
            else:
                all_overlaps = np.zeros((len(pspans), len(gspans)))
                for x1 in range(len(pspans)):
                    pspan = pspans[x1]
                    for x2 in range(len(gspans)):
                        gspan = gspans[x2]
                        start_ix = max(pspan[0], gspan[0])
                        end_ix = min(pspan[1], gspan[1])
                        overlap = max(0, end_ix - start_ix + 1)
                        if overlap > 0:
                            o1 = overlap / (pspan[1] - pspan[0] + 1)
                            o2 = overlap / (gspan[1] - gspan[0] + 1)
                            if min(o1, o2) >= .5:
                                all_overlaps[x1, x2] = max(o1, o2)
                unused_p_ix = set(range(len(pspans)))
                unused_g_ix = set(range(len(gspans)))
                col_size = len(pspans)
                row_size = len(gspans)
                for ix in np.argsort(all_overlaps.ravel())[::-1]:
                    if not len(unused_g_ix) or not len(unused_p_ix) or all_overlaps.ravel()[ix] == 0:
                        match_stats[cat_ix]['fp'] += len(unused_p_ix)
                        false_positives[cat_ix] = [(filtered_entities[cat_ix][p_ix][1:])
                                                    for p_ix in unused_p_ix]
                        match_stats[cat_ix]['fn'] += len(unused_g_ix)
                        break
                    p_ix = ix // row_size
                    g_ix = ix % row_size
                    if p_ix not in unused_p_ix or g_ix not in unused_g_ix:
                        continue
                    match_stats[cat_ix]['tp'] += 1
                    true_positives[cat_ix].append((filtered_entities[cat_ix][p_ix][1:]))
                    unused_g_ix.remove(g_ix)
                    unused_p_ix.remove(p_ix)


        return match_stats, num_entities, true_positives, false_positives


    def map_span_to_word_indices(span, index_map, bounds):
        return (index_map[bounds[span[0], 0]], index_map[bounds[span[1], 1] - 1])

    def split_predstring(x):
        vals = x.split()
        return int(vals[0]), int(vals[-1])

    def evaluate_score_thresholds(sorted_labels, tp, fp, fn):
        base_f1 = (f1(tp, fp, fn))
        scores = [base_f1]
        for ix in range(len(sorted_labels) - 1):
            if sorted_labels[ix] == 0:
                fp -= 1
            else:
                tp -= 1
                fn += 1
            scores.append(f1(tp, fp, fn))
            if scores[-1] < base_f1 - .01:
                break
        return scores

    def f1(tp, fp, fn):
        return tp / (tp + .5 * (fp + fn))

    def make_gt_dict(df):
        gt_dict = {}
        for cat_ix in range(1, 8):
            cat_name = label_names[cat_ix]
            cat_entities = df.loc[df.discourse_type==cat_name]
            if len(cat_entities):
                gt_dict[cat_ix] = [(x[0], x[1]) for x in cat_entities.predictionstring.map(split_predstring)]
        return gt_dict

    def f1s(stats):
        f1s = np.zeros(8)
        f1s[1:] = stats[:, 0] / (1e-7 + stats[:, 0] + stats[:, 1:].mean(-1))
        f1s[0] = f1s[1:].mean()
        return f1s

    def calc_entity_score(span, ps, c):
        s, e = span
        score = (ps[s, c * 2 - 1] + ps[s + 1: e + 1, c * 2].sum())/(e - s + 1)
        return score

    with open('oof_ps/gt_dicts.pickle', 'rb') as f:
        gt_dicts = pickle.load(f)


    list_of_dir_names = list_of_models

    arrays = []

    for array_name in 'all_outs, all_bounds, all_token_nums, all_word_indices, all_sample_ids'.split(', '):
        if array_name == 'all_outs':
            logps = np.load(f'oof_ps/{list_of_dir_names[0]}/{array_name}.npy', allow_pickle=True)
            arrays.append(np.exp(logps))

        else:
            arrays.append(np.load(f'oof_ps/{list_of_dir_names[0]}/{array_name}.npy', allow_pickle=True))

    all_outs, all_bounds, all_token_nums, all_word_indices, all_sample_ids = arrays
    all_outs = all_outs
    for ix in range(1, len(list_of_dir_names)):
        arrays = []
        for array_name in 'all_outs, all_bounds, all_token_nums'.split(', '):
            if array_name == 'all_outs':
                logps = np.load(f'oof_ps/{list_of_dir_names[ix]}/{array_name}.npy', allow_pickle=True)
                arrays.append(np.exp(logps))
            else:
                arrays.append(np.load(f'oof_ps/{list_of_dir_names[ix]}/{array_name}.npy', allow_pickle=True))
        new_outs, new_bounds, new_token_nums = arrays
        new_outs = new_outs
        merged_outs = []
        merged_bounds = []
        merged_token_nums = []
        for sample_ix in range(len(new_outs)):
            logits, bounds = merge_ps(all_outs[sample_ix][:all_token_nums[sample_ix]],
                                      new_outs[sample_ix][:new_token_nums[sample_ix]],
                                      all_bounds[sample_ix][:all_token_nums[sample_ix]],
                                      new_bounds[sample_ix][:new_token_nums[sample_ix]])
            merged_outs.append(logits)
            merged_bounds.append(bounds)
            merged_token_nums.append(len(logits))
        all_outs = merged_outs
        all_bounds = merged_bounds
        all_token_nums = merged_token_nums
    sample_id_to_ix = {x: ix for ix, x in enumerate(all_sample_ids)}


    label_names = ['None', 'Lead', 'Position', 'Evidence', 'Claim',
                   'Concluding Statement', 'Counterclaim', 'Rebuttal']

    len_exp = .2

    def extend_tokens(ent, n):
        return ent

    all_unfiltered_entities = [extract_entities(all_outs[sample_ix], all_token_nums[sample_ix])
                               for sample_ix in range(len(all_outs))]
    for sample_ix, sample_entities in enumerate(all_unfiltered_entities):
        for cat_ix in sample_entities:
            spans_with_scores_and_len = []
            for span in sample_entities[cat_ix]:
                spans_with_scores_and_len.append((span, calc_entity_score(span, all_outs[sample_ix], cat_ix) * (span[1] - span[0] + 1) ** len_exp, 
                                                 span[1] - span[0] + 1))
            all_unfiltered_entities[sample_ix][cat_ix] = spans_with_scores_and_len
    max_scores_hist = np.zeros((25, 8))
    score_thresholds_hist = np.zeros((25, 8))
    for min_num_tokens in range(25):
        token_len_filters, word_len_filters = ([min_num_tokens] * 8, [min_num_tokens] * 8)
        score_filters = [-100] * 8
        all_fps = {ix: [] for ix in range(1, 8)}
        all_tps = {ix: [] for ix in range(1, 8)}
        all_stats = np.zeros((len(val0_ids), 7, 3))
        for array_pos, sample_id in enumerate(val0_ids):
            sample_ix = sample_id_to_ix[sample_id]
            sample_stats, _, tp, fp  = process_sample(all_outs[sample_ix], all_unfiltered_entities[sample_ix],
                                                      all_word_indices[sample_ix], 
                                        all_bounds[sample_ix],
                                        gt_dicts[all_sample_ids[sample_ix]],
                                        all_token_nums[sample_ix],
                                        {ix:  {'fp': 0, 'fn': 0, 'tp': 0} for ix in range(1, 8)})
            for cat_ix in range(7):
                all_stats[array_pos, cat_ix] = [sample_stats[cat_ix + 1][x] for x in ('tp', 'fp', 'fn')]
                all_fps[cat_ix + 1].extend(fp[cat_ix + 1])
                all_tps[cat_ix + 1].extend(tp[cat_ix + 1])
        max_scores = np.zeros(8)
        thresholds = np.zeros(7)
        for cat_ix in range(1, 8):
            tp, fp, fn = all_stats[:, cat_ix - 1].sum(0)
            all_scores = [x[0] for x in all_tps[cat_ix]]
            all_scores.extend((x[0] for x in all_fps[cat_ix]))
            all_num_tokens = np.array([x[1] for x in all_tps[cat_ix]] + [x[1] for x in all_fps[cat_ix]])
            all_labels = [1] * len(all_tps[cat_ix]) + [0] * len(all_fps[cat_ix])
            all_scores = np.array(all_scores)
            all_labels = np.array(all_labels)
            sorted_index = np.argsort(all_scores)
            sorted_scores = all_scores[sorted_index]
            sorted_labels = all_labels[sorted_index]
            sorted_num_tokens = all_num_tokens[sorted_index]
            all_thresholds = np.r_[[sorted_scores[0]], .5 * (sorted_scores[1:] + sorted_scores[:-1])]
            evaluation_scores = evaluate_score_thresholds(sorted_labels, *all_stats.sum(0)[cat_ix - 1])
            conv_filter = np.ones(max(int(len(evaluation_scores) * .05) // 2 * 2, 2))
            half_len = len(conv_filter) // 2
            conv_filter *= (1/len(conv_filter))
            conved = np.convolve(evaluation_scores, conv_filter, 'same')
            conved[:half_len] /= np.arange(half_len, len(conv_filter)) * (1/len(conv_filter))
            conved[-half_len:] /= (np.arange(half_len + 1, len(conv_filter) + 1) * (1/len(conv_filter)))[::-1]
            max_ix = np.argmax(conved)
            max_scores[cat_ix] = conved[max_ix]
            thresholds[cat_ix - 1] = all_thresholds[max_ix]

            score_filtered_tp = sum(1 for x in all_tps[cat_ix] if x[0] <= thresholds[cat_ix - 1])
            score_filtered_fp = sum(1 for x in all_fps[cat_ix] if x[0] <= thresholds[cat_ix - 1])
            new_tp = tp - score_filtered_tp
            new_fp = fp - score_filtered_fp
            new_fn = fn + score_filtered_tp

            score_filtered_mask = sorted_scores > thresholds[cat_ix - 1]
            score_filtered_token_counts = sorted_num_tokens[score_filtered_mask]
            score_filtered_word_counts = sorted_num_tokens[score_filtered_mask]
            score_filtered_labels = sorted_labels[score_filtered_mask]

            score_filtered_f1 = new_tp / (new_tp + .5 * (new_fp + new_fn))
            max_scores_hist[min_num_tokens, cat_ix] = score_filtered_f1
            score_thresholds_hist[min_num_tokens, cat_ix] = thresholds[cat_ix - 1]

    token_len_filters = max_scores_hist.argmax(0)[1:]
    score_filters = score_thresholds_hist[max_scores_hist.argmax(0), np.arange(8)][1:]

    all_stats = np.zeros((len(val1_ids), 7, 3))
    for array_pos, sample_id in enumerate(val1_ids):
        sample_ix = sample_id_to_ix[sample_id]
        sample_stats, _, tp, fp  = process_sample(all_outs[sample_ix], all_unfiltered_entities[sample_ix],
                                                  all_word_indices[sample_ix], 
                                    all_bounds[sample_ix],
                                    gt_dicts[all_sample_ids[sample_ix]],
                                    all_token_nums[sample_ix],
                                    {ix:  {'fp': 0, 'fn': 0, 'tp': 0} for ix in range(1, 8)})
        for cat_ix in range(7):
            all_stats[array_pos, cat_ix] = [sample_stats[cat_ix + 1][x] for x in ('tp', 'fp', 'fn')]


    def extend_tokens(ent, n):
        nonlocal extension_sizes
        nonlocal extension_sizes2
        ent_size = ent[1] - ent[0] + 1
        for ix in range(len(extension_sizes2) - 1):
            if ent_size <= ix + 2:
                return [max(0, ent[0] + extension_sizes2[ix]) , min(n-1, ent[1] + extension_sizes[ix])]
        return [max(0, ent[0] + extension_sizes2[-1]) , min(n-1, ent[1] + extension_sizes[-1])]

    extension_sizes2 = [0] * 15
    results = np.zeros((15, 10, 8))
    for ext_ix in range(15):
        for ext_num in range(10):
            extension_sizes = [0] * 15
            extension_sizes[ext_ix] = ext_num
            all_stats = np.zeros((val_bounds[-1], 7, 3))
            for array_pos, sample_id in enumerate(val0_ids):
                sample_ix = sample_id_to_ix[sample_id]
                sample_stats, _, tp, fp  = process_sample(all_outs[sample_ix], all_unfiltered_entities[sample_ix],
                                                          all_word_indices[sample_ix], 
                                            all_bounds[sample_ix],
                                            gt_dicts[all_sample_ids[sample_ix]],
                                            all_token_nums[sample_ix],
                                            {ix:  {'fp': 0, 'fn': 0, 'tp': 0} for ix in range(1, 8)})
                for cat_ix in range(7):
                    all_stats[sample_ix, cat_ix] = [sample_stats[cat_ix + 1][x] for x in ('tp', 'fp', 'fn')]
            results[ext_ix, ext_num] = f1s(all_stats.sum(0))

    extension_sizes = results[..., 0].argmax(1)

    max_scores_hist = np.zeros((25, 8))
    score_thresholds_hist = np.zeros((25, 8))
    for min_num_tokens in range(25):
        token_len_filters, word_len_filters = ([min_num_tokens] * 8, [min_num_tokens] * 8)
        score_filters = [-100] * 8
        all_fps = {ix: [] for ix in range(1, 8)}
        all_tps = {ix: [] for ix in range(1, 8)}
        all_stats = np.zeros((len(val0_ids), 7, 3))
        for array_pos, sample_id in enumerate(val0_ids):
            sample_ix = sample_id_to_ix[sample_id]
            sample_stats, _, tp, fp  = process_sample(all_outs[sample_ix], all_unfiltered_entities[sample_ix],
                                                      all_word_indices[sample_ix], 
                                        all_bounds[sample_ix],
                                        gt_dicts[all_sample_ids[sample_ix]],
                                        all_token_nums[sample_ix],
                                        {ix:  {'fp': 0, 'fn': 0, 'tp': 0} for ix in range(1, 8)})
            for cat_ix in range(7):
                all_stats[array_pos, cat_ix] = [sample_stats[cat_ix + 1][x] for x in ('tp', 'fp', 'fn')]
                all_fps[cat_ix + 1].extend(fp[cat_ix + 1])
                all_tps[cat_ix + 1].extend(tp[cat_ix + 1])
        max_scores = np.zeros(8)
        thresholds = np.zeros(7)
        for cat_ix in range(1, 8):
            tp, fp, fn = all_stats[:, cat_ix - 1].sum(0)
            all_scores = [x[0] for x in all_tps[cat_ix]]
            all_scores.extend((x[0] for x in all_fps[cat_ix]))
            all_num_tokens = np.array([x[1] for x in all_tps[cat_ix]] + [x[1] for x in all_fps[cat_ix]])
            all_labels = [1] * len(all_tps[cat_ix]) + [0] * len(all_fps[cat_ix])
            all_scores = np.array(all_scores)
            all_labels = np.array(all_labels)
            sorted_index = np.argsort(all_scores)
            sorted_scores = all_scores[sorted_index]
            sorted_labels = all_labels[sorted_index]
            sorted_num_tokens = all_num_tokens[sorted_index]
            all_thresholds = np.r_[[sorted_scores[0]], .5 * (sorted_scores[1:] + sorted_scores[:-1])]
            evaluation_scores = evaluate_score_thresholds(sorted_labels, *all_stats.sum(0)[cat_ix - 1])
            conv_filter = np.ones(max(int(len(evaluation_scores) * .05) // 2 * 2, 2))
            half_len = len(conv_filter) // 2
            conv_filter *= (1/len(conv_filter))
            conved = np.convolve(evaluation_scores, conv_filter, 'same')
            conved[:half_len] /= np.arange(half_len, len(conv_filter)) * (1/len(conv_filter))
            conved[-half_len:] /= (np.arange(half_len + 1, len(conv_filter) + 1) * (1/len(conv_filter)))[::-1]
            max_ix = np.argmax(conved)
            max_scores[cat_ix] = conved[max_ix]
            thresholds[cat_ix - 1] = all_thresholds[max_ix]

            score_filtered_tp = sum(1 for x in all_tps[cat_ix] if x[0] <= thresholds[cat_ix - 1])
            score_filtered_fp = sum(1 for x in all_fps[cat_ix] if x[0] <= thresholds[cat_ix - 1])
            new_tp = tp - score_filtered_tp
            new_fp = fp - score_filtered_fp
            new_fn = fn + score_filtered_tp

            score_filtered_mask = sorted_scores > thresholds[cat_ix - 1]
            score_filtered_token_counts = sorted_num_tokens[score_filtered_mask]
            score_filtered_word_counts = sorted_num_tokens[score_filtered_mask]
            score_filtered_labels = sorted_labels[score_filtered_mask]

            score_filtered_f1 = new_tp / (new_tp + .5 * (new_fp + new_fn))
            max_scores_hist[min_num_tokens, cat_ix] = score_filtered_f1
            score_thresholds_hist[min_num_tokens, cat_ix] = thresholds[cat_ix - 1]


    token_len_filters = max_scores_hist.argmax(0)[1:]
    score_filters = score_thresholds_hist[max_scores_hist.argmax(0), np.arange(8)][1:]

    all_stats = np.zeros((len(val1_ids), 7, 3))
    for array_pos, sample_id in enumerate(val1_ids):
        sample_ix = sample_id_to_ix[sample_id]
        sample_stats, _, tp, fp  = process_sample(all_outs[sample_ix], all_unfiltered_entities[sample_ix],
                                                  all_word_indices[sample_ix], 
                                    all_bounds[sample_ix],
                                    gt_dicts[all_sample_ids[sample_ix]],
                                    all_token_nums[sample_ix],
                                    {ix:  {'fp': 0, 'fn': 0, 'tp': 0} for ix in range(1, 8)})
        for cat_ix in range(7):
            all_stats[array_pos, cat_ix] = [sample_stats[cat_ix + 1][x] for x in ('tp', 'fp', 'fn')]
    return f1s(all_stats.sum(0)), token_len_filters, score_filters, extension_sizes



In [ ]:
f1s, len_filters, score_filters, exts = evaluate_combination(['debertav2_xlarge',
                                                              'dirgyv3/clean',
                                                              'debertav1_xlarge/bugged',
                                                              'distilled_v1/clean'])

In [ ]:
print(f1s[0])#score